<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Guided Problem Set 11: Higgs Search Part II</h1>


<a name='section_11_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P11.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_11_1">P11.1 Fitting the Higgs</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_11_1">P11.1 Problems</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_11_2">P11.2 Using the Simulation</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_11_2">P11.2 Problems</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_11_3">P11.3 Hypothesis Test</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_11_3">P11.3 Problems</a></td>
    </tr>
</table>



<h3>Learning Objectives</h3>


In this Pset we continue our analysis of the Higgs data and ultimately perform a fit. Many of the required functions were introduced in the previous Pset. They are defined again in this preamble. Be sure to run all cells before continuing.

<h3>Data</h3>

>description: Higgs to 4 Leptons with CMS Open data from the Large Hadron Collider<br>
>source: https://zenodo.org/record/8034556 <br>
>attribution: Philip Harris (CMS Collaboration), DOI:10.5281/zenodo.8034556 


In [ ]:
#>>>RUN: P11.0-runcell00

!git init
!git remote add -f origin https://github.com/mitx-8s50/nb_LEARNER/
!git config core.sparseCheckout true
!echo 'P09' >> .git/info/sparse-checkout
!git pull origin main

In [ ]:
#>>>RUN: P11.0-runcell01

# NOTE: some files are too large to include in the original repository, so you must download them from here:
# https://www.dropbox.com/sh/wf7d42jg4mfbdlh/AAAXOXeEO9Bl_W6BUhV61kFpa?dl=0
#
# Ways to download:
#     1. Copy/paste the link (replace =0 with =1 to download automatically)
#     2. Use the wget commands below (works in Colab, but you may need to install wget if using locally)
#
# Location of files:
#     Move the files to the directory data/P09/MC
#
# Using wget: (works in Colab)
#     Upon downloading, the code below will move them to the appropriate directory

!wget https://www.dropbox.com/sh/wf7d42jg4mfbdlh/AADnJ6f0zzYekvFunRpH13Gma/zzto2mu2e2011.csv?dl=0
!wget https://www.dropbox.com/sh/wf7d42jg4mfbdlh/AAAgZQdH4nWP9dYUmDLWXfhXa/zzto2mu2e2012.csv?dl=0
!wget https://www.dropbox.com/sh/wf7d42jg4mfbdlh/AADepq5NkiLgh0uaij8P9p1Ka/zzto4e2011.csv?dl=0
!wget https://www.dropbox.com/sh/wf7d42jg4mfbdlh/AAA4FHGuNlPcfXHgwCcqZAyva/zzto4e2012.csv?dl=0
!wget https://www.dropbox.com/sh/wf7d42jg4mfbdlh/AAAxiJAH_89oEYYvfMpFdtq4a/zzto4mu2011.csv?dl=0
!wget https://www.dropbox.com/sh/wf7d42jg4mfbdlh/AAD6lR-feTLWa1-GvYCsy8N5a/zzto4mu2012.csv?dl=0
!mv zzto2mu2e2011.csv?dl=0 data/P09/MC/zzto2mu2e2011.csv
!mv zzto2mu2e2012.csv?dl=0 data/P09/MC/zzto2mu2e2012.csv
!mv zzto4e2011.csv?dl=0 data/P09/MC/zzto4e2011.csv
!mv zzto4e2012.csv?dl=0 data/P09/MC/zzto4e2012.csv
!mv zzto4mu2011.csv?dl=0 data/P09/MC/zzto4mu2011.csv
!mv zzto4mu2012.csv?dl=0 data/P09/MC/zzto4mu2012.csv

<h3>Importing Libraries</h3>

Before beginning, run the cell below to import the relevant libraries for this notebook.

In [ ]:
#>>>RUN: P11.0-runcell02

!pip install lmfit
!pip install dcor

In [ ]:
#>>>RUN: P11.0-runcell03

import numpy as np
import pandas as pd
import math as mt
import scipy.optimize as opt
import scipy.stats as sta
import matplotlib.pyplot as plt
import csv

from scipy.optimize import curve_fit
import scipy
# from scipy.stats import norm
from scipy import stats
import lmfit

<h3>Setting Default Figure Parameters</h3>

The following code cell sets default values for figure parameters.

In [ ]:
#>>>RUN: P11.0-runcell04

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title


<h3>Loading Data and Defining Relevant Functions Used Previously</h3>

Run all cells below before continuing. You will have to complete some cells based on your work in previous Psets. These cells will be indicated below.

In [ ]:
#>>>RUN: P11.0-runcell05

## some scale factors and constants relevant to the data

## mass squared of muon: m_muon^2 (in GeV)
sqm1 = (0.105658) * (0.105658)
## mass squared of electron: m_e^2 (in GeV)
sqme = (0.0005109989) * (0.0005109989)
## mass of Z boson (in GeV)
mZ = 91.1876


## constants for the scale factor of MC. The MC is produced at a certain cross section
## with a certain number of events. To increase statistics, the event numbers generated are usually very large,
## therefore a scale factor is needed to apply on the MC to make them agree with data.
## Scale factor for each MC component is: lumi * xsec / nevt, as provided below

## Luminosity of each year in pb
lumi12 = 11580.
lumi11 = 2330.

## MC cross section of each process
xsecZZ412 = 0.107
xsecZZ2mu2e12 = 0.249
xsecZZ411 = 0.093
xsecZZ2mu2e11 = 0.208

xsecTTBar12 = 200.
xsecTTBar11 = 19.504

xsecDY5012 = 2955.
xsecDY1012 = 10.742
xsecDY5011 = 2475.
xsecDY1011 = 9507.
  
scalexsecHZZ12 = 0.0065
scalexsecHZZ11 = 0.0057

## Number of MC Events generated for each process
nevtZZ4mu12 = 1499064
nevtZZ4e12 = 1499093
nevtZZ2mu2e12 = 1497445
nevtHZZ12 = 299973 
nevtTTBar12 = 6423106
nevtDY5012 = 29426492
nevtDY1012 = 6462290
  
nevtZZ4mu11 = 1447136
nevtZZ4e11 = 1493308
nevtZZ2mu2e11 = 1479879
nevtHZZ11 = 299683
nevtTTBar11 = 9771205
nevtDY5011 = 36408225
nevtDY1011 = 39909640


labels = ['$m_{H}$ = 125 GeV', r'ZZ $\rightarrow$ 4l', 'Z/$\gamma^{*}$ + X', r'$t\bar{t}$']
colors = ['r','b','g','gray']

#now we compute the event weights
scales_higgs = [lumi11*scalexsecHZZ11/nevtHZZ11, lumi12*scalexsecHZZ12/nevtHZZ12]
scales_zz = [lumi11*xsecZZ411/nevtZZ4mu11, lumi11*xsecZZ2mu2e11/nevtZZ2mu2e11, lumi11*xsecZZ411/nevtZZ4e11,\
             lumi12*xsecZZ412/nevtZZ4mu12, lumi12*xsecZZ2mu2e12/nevtZZ2mu2e12, lumi12*xsecZZ412/nevtZZ4e12]
scales_dy = [lumi11*xsecDY1011/nevtDY1011, lumi11*xsecDY5011/nevtDY5011, \
             lumi12*xsecDY1012/nevtDY1012, lumi11*xsecDY5012/nevtDY5012]
scales_tt = [lumi11*xsecTTBar11/nevtTTBar11, lumi12*xsecTTBar12/nevtTTBar12]

In [ ]:
#>>>RUN: P11.0-runcell06

### open the files ###
# read datalist of each year and combine to one 

def loaddata():
    data_year  = [pd.read_csv('data/P09/clean_data_2011.csv',index_col=None, header=0)]
    data_year += [pd.read_csv('data/P09/clean_data_2012.csv',index_col=None, header=0)]
    pdata = pd.concat(data_year,axis=0,ignore_index=True)
    return pdata

data=loaddata()

In [ ]:
#>>>RUN: P11.0-runcell07

## ZZ*
#NOTE: these zz* files are too large to include the original repository, so you must download them here:
#https://www.dropbox.com/sh/wf7d42jg4mfbdlh/AAAXOXeEO9Bl_W6BUhV61kFpa?dl=0
#Run cell P11.6-runcell01 to do this

def loadMC():
    # read MClist of each process and each year
    mc_higgs_11 = pd.read_csv('data/P09/MC/higgs2011.csv',index_col=None, header=0)
    mc_higgs_12 = pd.read_csv('data/P09/MC/higgs2012.csv',index_col=None, header=0)
    ## Drell-Yan
    mc_dy10_11 = pd.read_csv('data/P09/MC/dy1050_2011.csv',index_col=None, header=0)
    mc_dy50_11 = pd.read_csv('data/P09/MC/dy50_2011.csv',index_col=None, header=0)
    mc_dy10_12 = pd.read_csv('data/P09/MC/dy1050_2012.csv',index_col=None, header=0)
    mc_dy50_12 = pd.read_csv('data/P09/MC/dy50_2012.csv',index_col=None, header=0)
    ## ttbar
    mc_ttbar_11 = pd.read_csv('data/P09/MC/ttbar2011.csv',index_col=None, header=0)
    mc_ttbar_12 = pd.read_csv('data/P09/MC/ttbar2012.csv',index_col=None, header=0)
    ##zz
    mc_zz4mu_11 = pd.read_csv('data/P09/MC/zzto4mu2011.csv',index_col=None, header=0)
    mc_zz2mu2e_11 = pd.read_csv('data/P09/MC/zzto2mu2e2011.csv',index_col=None, header=0)
    mc_zz4e_11 = pd.read_csv('data/P09/MC/zzto4e2011.csv',index_col=None, header=0)
    mc_zz4mu_12 = pd.read_csv('data/P09/MC/zzto4mu2012.csv',index_col=None, header=0)
    mc_zz2mu2e_12 = pd.read_csv('data/P09/MC/zzto2mu2e2012.csv',index_col=None, header=0)
    mc_zz4e_12 = pd.read_csv('data/P09/MC/zzto4e2012.csv',index_col=None, header=0)

    # create a combined list of MC
    mc_higgs = [mc_higgs_11, mc_higgs_12]
    mc_zz = [mc_zz4mu_11, mc_zz2mu2e_11, mc_zz4e_11, mc_zz4mu_12, mc_zz2mu2e_12, mc_zz4e_12]
    mc_dy = [mc_dy10_11, mc_dy50_11, mc_dy10_12, mc_dy50_12]
    mc_tt = [mc_ttbar_11, mc_ttbar_12]

    out_mc_sig = pd.concat(mc_higgs,axis=0,ignore_index=True)
    out_mc_bkg_zz = pd.concat(mc_zz,axis=0,ignore_index=True)
    out_mc_bkg_dy = pd.concat(mc_dy,axis=0,ignore_index=True)
    out_mc_bkg_tt = pd.concat(mc_tt,axis=0,ignore_index=True)

    ## For Selections:
    out_mc_all = [mc_higgs, mc_zz, mc_dy, mc_tt]
    return out_mc_all,out_mc_sig,out_mc_bkg_zz,out_mc_bkg_dy,out_mc_bkg_tt

mc_all,mc_sig,mc_bkg_zz,mc_bkg_dy,mc_bkg_tt=loadMC()

In [ ]:
#>>>RUN: P11.0-runcell08

#This is the header of the csv file:
#Run,Event,PID1,Q1,E1,px1,py1,pz1,eta1,phi1,PID2,Q2,E2,px2,py2,pz2,eta2,phi2,PID3,Q3,E3,px3,py3,pz3,eta3,phi3,PID4,Q4,E4,px4,py4,pz4,eta4,phi4
def ReadEntries(lis):
    return lis['PID1'],lis['Q1'],lis['E1'],lis['px1'],lis['py1'],lis['pz1'],lis['eta1'],lis['phi1'], \
    lis['PID2'],lis['Q2'],lis['E2'],lis['px2'],lis['py2'],lis['pz2'],lis['eta2'],lis['phi2'], \
    lis['PID3'],lis['Q3'],lis['E3'],lis['px3'],lis['py3'],lis['pz3'],lis['eta3'],lis['phi3'], \
    lis['PID4'],lis['Q4'],lis['E4'],lis['px4'],lis['py4'],lis['pz4'],lis['eta4'],lis['phi4']

def pt(px, py):
    return np.sqrt(px**2 + py**2)

def invMass(E, px, py, pz):
    return np.sqrt(E**2 - (px**2 + py**2 + pz**2))

def InvMass_4l(lists):   ### faster way to get the 4l-system invMass directly from the list, as you may repeately run it for the final plot###
    E_tot = lists['E1'] + lists['E2'] + lists['E3'] + lists['E4']
    px_tot = lists['px1'] + lists['px2'] + lists['px3'] + lists['px4']
    py_tot = lists['py1'] + lists['py2'] + lists['py3'] + lists['py4']
    pz_tot = lists['pz1'] + lists['pz2'] + lists['pz3'] + lists['pz4']
    return np.sqrt(E_tot**2 - (px_tot**2 + py_tot**2 + pz_tot**2))

In [ ]:
#>>>RUN: P11.0-runcell09

## function for making histograms of different data features
def PlotShape(variable, rmin, rmax, nbins):
    for i,var in enumerate(variable):
        hist, bins = np.histogram(var, bins = nbins, range = (rmin,rmax))
        bincenter = 0.5 * (bins[:-1] + bins[1:])
        plt.plot(bincenter, hist * (1./len(var)), color = colors[i], label=labels[i],drawstyle = 'steps-mid')
    plt.legend(fontsize = 12)

<h3>Complete Cell Below Based on Previous Work</h3>

In [ ]:
#>>>RUN: P11.0-runcell10

def select_pairs(PID1,PID2,PID3,m_z1_c1,m_z2_c1,m_z1_c2,m_z2_c2,m_z1_c3,m_z2_c3,upperMinMass,lowerMinMass):
    ### 2mu2e event, only one combination c3
        
    ### 2mu2e event, only one combination c2

    ### 4mu or 4e event, 3 different combination

    return pass

In [ ]:
#>>>RUN: P11.0-runcell11

def obj(pid,px,py,eta):  ## object selection for a single object
    pt_obj = pt(px,py)
    if (np.abs(pid) == 13 and pt_obj > 5.) or (np.abs(pid) == 11 and pt_obj > 7.):## requires electrons and muons to have transverse momenta above certain level
        return 1
    else:
        return 0
    
def objsel(lis):  ## Create column indicating whether this event passes the object selection
    pass_obj = []
    for row in lis.itertuples():   ## loop over each event(each row)
        pass_obj.append(obj(row.PID1,row.px1,row.py1,row.eta1) and obj(row.PID2,row.px2,row.py2,row.eta2) and obj(row.PID3,row.px3,row.py3,row.eta3) and obj(row.PID4,row.px4,row.py4,row.eta4))
    lis['PassObj'] = pass_obj  

<h3>Complete Cell Below Based on Previous Work</h3>

In [ ]:
#>>>RUN: P11.0-runcell12
 
def evt(PID1,Q1,E1,px1,py1,pz1,PID2,Q2,E2,px2,py2,pz2,PID3,Q3,E3,px3,py3,pz3,PID4,Q4,E4,px4,py4,pz4):
    
    if PID1+PID2+PID3+PID4 != 0: #Charge conservation
        return 0
    
    ### All Pairwise Mass combinations
    ##c1
    m_z1_c1 = np.sqrt((E1+E2)**2 - ((px1+px2)**2 + (py1+py2)**2 + (pz1+pz2)**2))
    m_z2_c1 = np.sqrt((E3+E4)**2 - ((px3+px4)**2 + (py3+py4)**2 + (pz3+pz4)**2))
    ##c2
    m_z1_c2 = np.sqrt((E1+E3)**2 - ((px1+px3)**2 + (py1+py3)**2 + (pz1+pz3)**2))
    m_z2_c2 = np.sqrt((E2+E4)**2 - ((px2+px4)**2 + (py2+py4)**2 + (pz2+pz4)**2))
    ##c3
    m_z1_c3 = np.sqrt((E1+E4)**2 - ((px1+px4)**2 + (py1+py4)**2 + (pz1+pz4)**2))
    m_z2_c3 = np.sqrt((E2+E3)**2 - ((px2+px3)**2 + (py2+py3)**2 + (pz2+pz3)**2))

    ###NOTE: IF YOU DID NOT PREVIOUSLY DEFINE THE `lowerMinMass` and `upperMinMass`, do so here
    lowerMinMass = 0 #YOUR VALUE HERE
    upperMinMass = 0 #YOUR VALUE HERE

    if select_pairs(PID1,PID2,PID3,m_z1_c1,m_z2_c1,m_z1_c2,m_z2_c2,m_z1_c3,m_z2_c3,upperMinMass,lowerMinMass) == 0:
      return 0
        
    return 1    
    
    
def evtsel(lis):   ### Similar as the Function: objsel shown above
    pass_evt = []
    for row in lis.itertuples():
        pass_evt.append(evt(row.PID1,row.Q1,row.E1,row.px1,row.py1,row.pz1,\
                            row.PID2,row.Q2,row.E2,row.px2,row.py2,row.pz2,\
                            row.PID3,row.Q3,row.E3,row.px3,row.py3,row.pz3,\
                            row.PID4,row.Q4,row.E4,row.px4,row.py4,row.pz4))
    lis['PassEvt'] = pass_evt

In [ ]:
#>>>RUN: P11.0-runcell13
 
### Apply object selection and event selection to data and each MC process###
### This will take a little time to run, about a minute###
### You may see Runtime warnings regarding "invalid value encountered in sqrt"###

#Select Data
objsel(data)
evtsel(data)
sel_data = data.query('PassObj == 1 and PassEvt == 1')

#Select MC
mc_all,mc_sig,mc_bkg_zz,mc_bkg_dy,mc_bkg_tt=loadMC()
for i,bkg in enumerate(mc_all):
    for j,pro in enumerate(bkg):
        objsel(pro)
        evtsel(pro)
        mc_all[i][j] = pro.query('PassObj == 1 and PassEvt == 1')

In [ ]:
#>>>RUN: P11.0-runcell14

def HistInvMass4l(lis, scales, rmin, rmax, nbins):
    hist_lis = []
    for i,pro in enumerate(lis):
        invM = InvMass_4l(pro)
        hist, bins = np.histogram(invM, bins = nbins, range = (rmin,rmax))
        hist = hist * scales[i]
        hist_lis.append(hist)
    return sum(hist_lis)

def makeAllHists(idata,imc,rmin=50,rmax=200,nbins=50):
    hist_higgs = HistInvMass4l(imc[0],scales_higgs,rmin, rmax, nbins)
    hist_zz    = HistInvMass4l(imc[1],scales_zz,rmin, rmax, nbins)
    hist_dy    = HistInvMass4l(imc[2],scales_dy,rmin, rmax, nbins)
    hist_tt    = HistInvMass4l(imc[3],scales_tt,rmin, rmax, nbins)
    out_hist_mc    = [hist_tt, hist_dy, hist_zz, hist_higgs]

    inM_data = InvMass_4l(idata)
    out_hist_data, bins = np.histogram(inM_data, bins = nbins, range = (rmin,rmax))
    return out_hist_mc,out_hist_data

hist_mc,hist_data=makeAllHists(sel_data,mc_all)

In [ ]:
#>>>RUN: P11.0-runcell15

## Now Plot it!

def plotHists(ihist_mc,ihist_data,rmin=50,rmax=200,nbins=50):
    plt.figure(figsize = (8,6))
    # plt.style.use('mystyle.mplstyle')
    binwidth  = (rmax-rmin)//nbins
    bincenter = np.arange(rmin+binwidth*0.5,rmax+binwidth*0.5,binwidth)
    ## MC
    for i,pro in enumerate(ihist_mc):
        if i == 0:
            stack = plt.bar(bincenter, pro, align = 'center', width = binwidth, color = colors[3-i], label=labels[3-i],
            alpha = 0.5)
            bot = pro
        else:
            stack = plt.bar(bincenter, pro, align = 'center', width = binwidth, color = colors[3-i], label=labels[3-i],
                 bottom = bot, alpha = 0.5)
            bot = bot + pro

    ## Measured data
    xerrs = [binwidth*0.5 for i in range(0, nbins)]
    yerrs = np.sqrt(ihist_data)
    marker_data = plt.errorbar(bincenter, ihist_data, xerr = xerrs, yerr = yerrs, linestyle = 'None', color = 'black',
                        marker = 'o', label = 'Data')

    plt.title('$ \sqrt{s} = 7$ TeV, L = 2.3 $fb^{-1}$; $\sqrt{s} = 8$ TeV, L = 11.6 $fb^{-1}$ \n', fontsize = 15, position=(0.64,0.95))
    plt.xlabel('$m_{4l}$ [GeV]',fontsize = 20, position=(0.92,0.1))
    plt.ylabel('Events / 3 GeV',fontsize = 20, position=(0.1,0.84))
    plt.xlim(rmin,rmax)
    plt.legend(fontsize = 20)
    plt.show()

#plotHists(hist_mc,hist_data)

<a name='section_11_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P11.1 Fitting the Higgs Data</h2>   

| [Top](#section_11_0) | [Previous Section](#section_11_0) | [Problems](#problems_11_1) | [Next Section](#section_11_2) | 

<h3>Overview</h3>

Soon, you will be fitting mass distributions to make conclusions of the Higgs boson. Before you do that, consider a toy problem to get warmed up with using `lmfit` to fit distributions. 

<h3>Fitting a Binomial Distribution</h3>

One interesting result derived from statistics is that the limit of a binomial distribution is a normal distribution as the number of trials gets large. Usually, this is proved using pages of calculus. However, we have the tools to see it experimentally. In the following problem, we will fit a Gaussian distribution to randomly sampled data.

First, let's generates a binomial distribution with a given probability of success (`p=0.25`) and number of trials (`N=100`), and plot a histogram of the generated samples. 

In [ ]:
#>>>RUN: P11.1-runcell01

#set the random seed
np.random.seed(42)

N = 100 # number of trials

#Sample the the binomial
binom = np.random.binomial(p = 0.25, n=N, size=100000)

#Create the bin centers
nbins=100
binwidth  = (N - 0)/nbins

#create the histogram
bins = np.arange(0, N, binwidth)
hist = plt.hist(binom, bins=bins,  label = "n="+str(N)+" data")[0]
plt.show()

Now, create a fitter which fits this distribution to a normal distribution!

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 11.1.1</span>

Complete the code below to fit a normal distribution to the simulated data.

Does the normal distribution do a good job approximating the binomial sample for `N=100`? Does the normal distribution tend to overestimate or underestimate the number of samples at the upper end of the binomial distribution?

A) Yes, overestimate\
B) Yes, underestimate\
C) No, overestimate\
D) No, underestimate

In [ ]:
#>>>PROBLEM: P11.1.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def gauss1d(x,amp,x0,sigma):
    #create a Gaussian centered on x0,
    #with amplitude amp and width sigma
    return np.abs(amp)*np.exp(-(x-x0)**2/sigma**2)

def fitModel(iX,iY,iFunc, x0, sigma):
    model  = ##YOUR CODE HERE.    HINT: Use lmfit.Model, check lmfit documentation for help
    p =      ##YOUR CODE HERE.    HINT: Use model.make_params, check lmfit documentation for help
    result = ##YOUR CODE HERE.    HINT: Use model.fit, check lmfit documentation for help
    output = ##YOUR CODE HERE.    HINT: Use model.eval, check lmfit documentation for help
    return output,result

bincenter = np.arange(0+binwidth*0.5,N-binwidth*0.5,binwidth)
out,res = fitModel(bincenter, hist, gauss1d, N*0.25, np.sqrt(N*0.25*(1-0.25)))

#make the plots
#create the histogram
plt.hist(binom, bins=bins,  label = "n="+str(N)+" data")[0]
plt.plot(bincenter,out, label = "n="+str(N)+" model")

lmfit.report_fit(res)
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 11.1.2</span>

With that warm-up, let's return to the Higgs! Fit a Gaussian to only the MC Higgs peak. What is the mass and width of this feature in GeV, with 1e-1 precision? Enter your answer as a list: `[mass,width]`

Note: To answer this and the following questions, only include the selections already outlined in the code. Use your selections to improve upon these results after answering the questions.

In [ ]:
#>>>PROBLEM: P11.1.2
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

## fit the Higgs MC width and center to find a first guess for discovery analysis
rmin=50
rmax=200
nbins=50
binwidth  = (rmax-rmin)//nbins
bincenter = np.arange(rmin+binwidth*0.5,rmax+binwidth*0.5,binwidth)

higgsMC = hist_mc[3]
plt.plot(bincenter,higgsMC,drawstyle = 'steps-mid')

def gauss1d(x,amp,x0,sigma):
    #create a Gaussian centered on x0,
    #with amplitude amp and width sigma
    return np.abs(amp)*np.exp(-(x-x0)**2/sigma**2)

def fitModel(iX,iY,iFunc):
    model  = #YOUR CODE HERE
    p  =     #YOUR CODE HERE #USE x0=125,sigma=2
    result = #YOUR CODE HERE
    output = #YOUR CODE HERE
    return output,result

#we're fitting higgsMC with the function gauss1d
out,res = ##YOUR CODE HERE 
plt.plot(bincenter,out)
plt.xlabel('$m_{4l}$ [GeV]',fontsize = 20, position=(0.92,0.1))
plt.ylabel('Events / 3 GeV',fontsize = 20, position=(0.1,0.84))

for paramName, paramValue in res.params.items():
  print(paramName, paramValue)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 11.1.3</span>

Now take the whole selection and fit a linear polynomial+Gaussian for the signal to the Higgs selected data. The Gaussian should have amplitude `amp`, with mean `mh` and width `sigma`. When fitting the data, fix the width to sigma=3 GeV.

What is your signal significance? For now, just look at the significance of the fit parameter `amp`, and calculate the ration of `amp` divided by it's uncertainty. Thereby express your answer in terms of standard deviations, with precision 1e-1 (e.g., 1.1 stdev, 4.6 stdev, etc.).

To answer this and the following questions, only include the selections already outlined in the code. Use your selections to improve upon these results!

In [ ]:
#>>>PROBLEM: P11.1.3

def pol2(x, p0, p1,mh,sigma):
    #YOUR CODE HEREs
    return 

def sigpol2(x,p0,p1,amp,mh,sigma):
    bkg=pol2(x,p0,p1,mh,sigma)
    sig=#YOUR CODE HERE
    return sig+bkg

def fitModelMFix(iX,iY,iWeights,iFunc):
    model  = lmfit.Model(iFunc)
    #YOUR CODE HERE p = model.make_params(...)
    p["sigma"].vary=False
    result = model.fit(data=iY,params=p,x=iX,weights=iWeights)
    #now make a fine binned evaluation of the fit
    xinterp = np.arange(iX[0],iX[-1],0.1)
    output = model.eval(params=result.params,x=xinterp)
    return xinterp,output,result

def makeHistAndFit(ihist_data,rmin=50,rmax=200,nbins=50):
    binwidth    = (rmax-rmin)//nbins
    bincenter   =  np.arange(rmin+binwidth*0.5,rmax+binwidth*0.5,binwidth)
    #pre cut the small region for fitting
    smallrange  =  bincenter [(bincenter > 100) & (bincenter < 165) & (ihist_data > 0)]
    smallhist   =  ihist_data[(bincenter > 100) & (bincenter < 165) & (ihist_data > 0)]
    smallweight = (ihist_data[(bincenter > 100) & (bincenter < 165) & (ihist_data > 0)])**0.5
    #now fit
    xinterp,output,result2 = fitModelMFix(smallrange,smallhist,1./smallweight,sigpol2)
    #now plot
    plt.errorbar(smallrange,smallhist, yerr = smallweight, linestyle = 'None', color = 'black', marker = 'o', label = 'Data')
    plt.plot(xinterp,output)
    plt.xlabel('$m_{4l}$ [GeV]',fontsize = 20, position=(0.92,0.1))
    plt.ylabel('Events (scaled)',fontsize = 20, position=(0.1,0.84))
    lmfit.report_fit(result2)
    plt.show()

makeHistAndFit(hist_data)

<a name='section_11_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P11.2 Using the Simulation</h2>   

| [Top](#section_11_0) | [Previous Section](#section_11_1) | [Problems](#problems_11_2) | [Next Section](#section_11_3) | 

<h3>Overview</h3>

This fit looks ok, but we can do better using our simulation. Using the simulation, we can actually build functions by just throwing it to the data. To give you two ways to do this, let's first use a spline interpolated function. So what happened is, we just took the data and turned it into a smooth function that we can evaulate anywhere. This is done by chunking up the data into little pieces and fitting higher order polynomials to each. We end up with a function.

Since its a function, we can do whatever we want with it. Ok so lets go ahead and take our existing histograms and interpolate them so that we can turn them into a function. Let's go ahead and plot our histgrams and then turn them into functions.

In [ ]:
#>>>RUN: P11.2-runcell01

#load the interpolation package
from scipy import interpolate

def setupSpline(ihist_mc,rmin=50,rmax=200,nbins=50):
    binwidth    = (rmax-rmin)//nbins
    bincenter   =  np.arange(rmin+binwidth*0.5,rmax+binwidth*0.5,binwidth)
    sig=hist_mc[-1]
    ## Sum over all the backgrounds and plot at the same time
    sumbkg=0
    for i,pro in enumerate(ihist_mc):
        if i == 0:
            stack = plt.bar(bincenter, pro, align = 'center', width = binwidth, color = colors[3-i], label=labels[3-i],alpha = 0.5)
            bot = pro
        else:
            stack = plt.bar(bincenter, pro, align = 'center', width = binwidth, color = colors[3-i], label=labels[3-i],bottom = bot, alpha = 0.5)
            bot = bot + pro
        if i == 2:
            sumbkg = bot
    plt.bar(bincenter,sumbkg,align='center',width=binwidth)

    #spline interpolate
    out_bkg_interpolate = interpolate.splrep(bincenter, sumbkg)
    out_sig_interpolate = interpolate.splrep(bincenter, sig)

    #Now plot the splines as a function evaluation
    x_int = np.linspace(50, 200,1000)
    #this is the evalution
    y_bkg = interpolate.splev(x_int, out_bkg_interpolate)
    y_sig = interpolate.splev(x_int, out_sig_interpolate)
    plt.plot(x_int, y_bkg, 'b',label='bkg')
    plt.plot(x_int, y_sig+y_bkg, 'r',label='sig+bkg')
    #plt.plot(x,y_mc,drawstyle = 'steps-mid')
    plt.xlabel("$m_{4\ell}$(GeV)")
    plt.ylabel("$N_{events}$")
    plt.legend()
    plt.show()
    return out_sig_interpolate,out_bkg_interpolate

sig_interpolate,bkg_interpolate=setupSpline(hist_mc)

Now we have two functions from interpolating the MC result, we can treat them as functions. We have one function for signal and one function for background. First, let's see what happens when we fit the data with these new functions. This is not going to be the most realistic fit, but its a good example to start with.

In [ ]:
#>>>RUN: P11.2-runcell02

#ok lets setup the fit now that we have the ability to interpolate 
#as a first pass we will just fit the data with lmfit
import lmfit 

def sigbkg(x,bkgnorm=1,signorm=1):
    y_bkg = interpolate.splev(x, bkg_interpolate)
    y_sig = interpolate.splev(x, sig_interpolate)
    return y_bkg*bkgnorm + y_sig*signorm

def fitModel(iX,iY,iWeights,iFunc):
    model  = lmfit.Model(iFunc)
    p = model.make_params(bkgnorm=1,signnorm=1)
    result = model.fit(data=iY,params=p,x=iX,weights=iWeights)
    xinterp = np.arange(iX[0],iX[-1],0.1)
    output = model.eval(params=result.params,x=xinterp)
    return xinterp,output,result

def fithistograms(idata_hist,rmin=50,rmax=200,nbins=50):
    binwidth    = (rmax-rmin)//nbins
    bincenter   =  np.arange(rmin+binwidth*0.5,rmax+binwidth*0.5,binwidth)
    nonzerobin = bincenter[idata_hist > 0]
    nonzerohist= idata_hist[idata_hist > 0]
    error      = np.sqrt(idata_hist[idata_hist > 0])
    xinterp,output,result2 = fitModel(nonzerobin,nonzerohist,1./error,sigbkg)
    plt.errorbar(nonzerobin,nonzerohist, yerr = error, linestyle = 'None', color = 'black', marker = 'o', label = 'Data')
    plt.plot(xinterp,output)
    lmfit.report_fit(result2)
    plt.show()

fithistograms(hist_data)

The above fit to the data is not great. The strength of the signal in the fit is barely two sigma (taking the amplitude divided by the uncertainty: `1.25/0.55 = 2.27`).

So far, this fit enforces the data to have the same shape as the MC results because that is what the spline is based on. It is not fitting for a Higgs boson mass, it just fits for the strength of the signal amplitude. We can both improve the fit and get a first measure of the Higgs boson mass from our data selections by allowing the Higgs mass to vary!

The following function includes a variable shift in the x axis to allow for a varying Higgs mass. Check it out! We will use it in the following problems.

In [ ]:
#>>>RUN: P11.2-runcell03

def sig(x,mh):
    y_sig = interpolate.splev(x+(mh-125), sig_interpolate)
    return y_sig

xaxis=np.arange(90,150,0.1)
ydown=sig(xaxis,120)
ybase=sig(xaxis,125)
yup  =sig(xaxis,130)

plt.plot(xaxis,ybase,label='mh=125')
plt.plot(xaxis,yup,label='mh=130')
plt.plot(xaxis,ydown,label='mh=120')
plt.xlabel('m$_{4\ell}$')
plt.ylabel('Events')
plt.legend()
plt.show()


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 11.2.1</span>

What do you measure to be the Higgs boson mass in GeV, by allowing a shift in the x-axis of the fit function? Report your answer with precision 1e-2 (even though, really, we should use the precision that makes sense given the fit uncertainty).

In [ ]:
#>>>PROBLEM: P11.2.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def sigbkg(x,mh,shift,bkgnorm=1,signorm=1):
    y_bkg = #YOUR CODE HERE shift, shifts background
    y_sig = #YOUR CODE HERE mh moves signal like above
    return y_bkg*bkgnorm + y_sig*signorm

def fitModel(iX,iY,iWeights,iFunc):
    model  = lmfit.Model(iFunc)
    p = model.make_params(mh=125,shift=0,bkgnorm=1,signnorm=1)
    result = model.fit(data=iY,params=p,x=iX,weights=iWeights)
    xinterp = np.arange(iX[0],iX[-1],0.1)
    output = model.eval(params=result.params,x=xinterp)
    return xinterp,output,result

def fithistograms(idata_hist,ifunc,rmin=50,rmax=200,nbins=50):
    binwidth    = (rmax-rmin)//nbins
    bincenter   =  np.arange(rmin+binwidth*0.5,rmax+binwidth*0.5,binwidth)
    nonzerobin = bincenter[idata_hist > 0]
    nonzerohist= idata_hist[idata_hist > 0]
    error      = np.sqrt(idata_hist[idata_hist > 0])
    xinterp,output,result2 = fitModel(nonzerobin,nonzerohist,1./error,ifunc)
    plt.errorbar(nonzerobin,nonzerohist, yerr = error, linestyle = 'None', color = 'black', marker = 'o', label = 'Data')
    plt.plot(xinterp,output)
    lmfit.report_fit(result2)
    plt.show()
    return result2.chisqr

chi2sig=fithistograms(hist_data,sigbkg)
print(chi2sig)

<a name='section_11_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P11.3 Hypothesis Test</h2>   

| [Top](#section_11_0) | [Previous Section](#section_11_2) | [Problems](#problems_11_3) | 

<h3>Overview</h3>

We have performed a measurement of the mass of the Higgs boson! However, let's do this with the full hypothesis test. First, lets setup a background-only fit, without signal, and fit it to the data. 

In [ ]:
#>>>RUN: P11.3-runcell01

import scipy.optimize as optimize

def shiftbkg(x,mh,shift,bkgnorm=1):
    y_bkg = interpolate.splev(x+shift, bkg_interpolate)
    return y_bkg*bkgnorm

chi2bkg=fithistograms(hist_data,shiftbkg)

To see how significant this Higgs bump we have measured, let's compute delta chi2 and translate this to a p-value.

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 11.3.1</span>

Given the chi2 values for signal and background fits that we have found in the preceding work (i.e., `chi2sig` and `chi2bkg`), compute the following:

1) The negative delta log-likelihood value.\
2) The p-value corresponding to the delta $\chi^{2}$.\
3) The significance of a normal distribution, in terms of standard deviations, using the two sided significance.

Remember that we are floating mass and signal normalization in this fit so there are 2 degrees of freedom.

Report your answer as a list of numbers with precision 1e-3: `[NLL, pval, significance]`

In [ ]:
#>>>PROBLEM: P11.3.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def get_significance_vals(iChiSB,iChiB,iNDF=2):
    #YOUR CODE HERE
    return NLL,pval,sigma

NLL,pval,sigma=get_significance_vals(chi2sig,chi2bkg)
print("NLL",NLL,"pval",pval,"sig",sigma)

<h3>Making p-value plot</h3>

Ok, so now we want to make the traditional p-value plot. For this we are going to run our fit now with a fixed mass and then compute $\Delta\chi^{2}$ and translate this to a p-value. To do this we now need to make a fit with fixed mass.

In [ ]:
#>>>RUN: P11.3-runcell02

def fitModel(iM,iX,iY,iWeights,iFunc):
    model  = lmfit.Model(iFunc)
    p = model.make_params(mh=iM,shift=0,bkgnorm=1,signnorm=1)
    p['mh'].vary=False
    result = model.fit(data=iY,params=p,x=iX,weights=iWeights)
    xinterp = np.arange(iX[0],iX[-1],0.1)
    output = model.eval(params=result.params,x=xinterp)
    return xinterp,output,result

def fithistograms(iM,idata_hist,ifunc,rmin=50,rmax=200,nbins=50,plot=False):
    binwidth    = (rmax-rmin)//nbins
    bincenter   =  np.arange(rmin+binwidth*0.5,rmax+binwidth*0.5,binwidth)
    nonzerobin = bincenter[idata_hist > 0]
    nonzerohist= idata_hist[idata_hist > 0]
    error      = np.sqrt(idata_hist[idata_hist > 0])
    xinterp,output,result2 = fitModel(iM,nonzerobin,nonzerohist,1./error,ifunc)
    if plot:
        plt.errorbar(nonzerobin,nonzerohist, yerr = error, linestyle = 'None', color = 'black', marker = 'o', label = 'Data')
        plt.plot(xinterp,output)
        lmfit.report_fit(result2)
        plt.show()
    return result2.chisqr

fithistograms(125,hist_data,sigbkg,plot=True)

Now finally, we will scan the mass and make a p-value plot of the significance vs mass, can we see the Higgs? 

In [ ]:
#>>>RUN: P11.5-runcell03

def massscan(isigbkg,plot=True):
    mhscan = np.arange(110,150,.1)
    NLLs   = np.array([])
    pscan  = np.array([])
    sigscan  = np.array([])
    for pMH in mhscan: 
        chi2sig = fithistograms(pMH,hist_data,isigbkg)
        NLL,pval,sigma=get_significance_vals(chi2sig,chi2bkg,1)
        NLLs  = np.append(NLLs,-2*NLL)
        pscan = np.append(pscan,pval)
        sigscan = np.append(sigscan,sigma)
    
    NLLs = NLLs-np.min(NLLs)
    if plot:
        plt.plot(mhscan,NLLs)
        plt.xlabel("m$_{h}$ (125)")
        plt.ylabel("2 log($\Lambda_{1}/\Lambda_{2}$)")
        plt.show()

        plt.plot(mhscan,pscan)
        plt.xlabel("m$_{h}$ (125)")
        plt.ylabel("p")
        plt.show()
    
        plt.plot(mhscan,sigscan)
        plt.xlabel("m$_{h}$ (125)")
        plt.ylabel("$\sigma$")
        plt.show()
 
    return mhscan,NLLs,pscan

mhscan,NLLs,pscan=massscan(sigbkg)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 11.3.2</span>

What do you measure to be the best fit mass? What is the mass window from this analysis? Enter in the form `[lower end of mass range, upper end of mass range, best fit mass,]` with precision 1e-1 in GeV.

In [ ]:
#>>>PROBLEM P11.3.2

def mass(imscan, iNLLs,iMin=110,iMax=135):
    ##YOUR CODE HERE (if you want to do it with code)
    #uncomment line below and complete code
    #print("Mass window is:",  ,"Best fit:",  ,"+/-",  )
    return
    
mass(imscan, iNLLs,iMin=110,iMax=135)

<h3>Add Another Parameter</h3>

An additional parameter we can tune to make a precision fit is the width of the Higgs peak. Applying a numerical convolution to the fit shape with a Gaussian distribution smears it out, making it wider. Leaving the width of the Gaussian that is convolved with the interpolated function as a free parameter allows us to vary the width of the fitted Higgs peak. This what we will do below.

In [ ]:
#>>>RUN: P11.3-runcell04

import scipy.stats as stats

def conGaus(x,tck,sigma=1,mean=0,iMin=-3,iMax=3,iN=10):
    step=float((iMax-iMin))/float(iN)
    pInt=0
    for i0 in range(iN):
            pX   = (i0*step+iMin)*sigma
            pVal = interpolate.splev(x-pX,tck)*(stats.norm.pdf(pX,mean,sigma))
            pInt += pVal*step
    return pInt

def sigbkg(x,mh,shift,bkgnorm=1,signorm=1,sigma=1):
    y_bkg = conGaus(x+shift, bkg_interpolate,sigma)
    y_sig = conGaus(x+(mh-125), sig_interpolate,sigma)
    return y_bkg*bkgnorm + y_sig*signorm

def bkg(x,mh,shift,bkgnorm=1,sigma=1):
    y_bkg = conGaus(x+shift, bkg_interpolate,sigma)
    return y_bkg*bkgnorm

def fitModel(iM,iX,iY,iWeights,iFunc):
    model  = lmfit.Model(iFunc)
    p = model.make_params(mh=iM,shift=0,bkgnorm=1,signnorm=1,sigma=1)
    p['mh'].vary=False
    result = model.fit(data=iY,params=p,x=iX,weights=iWeights)
    xinterp = np.arange(iX[0],iX[-1],0.1)
    output = model.eval(params=result.params,x=xinterp)
    return xinterp,output,result

chi2bkg=fithistograms(125,hist_data,bkg,plot=True)
mhscan,NLLs,pscan=massscan(sigbkg,plot=True)


#see solution code in previous problem for definition of mass() function
mass(mhscan,NLLs,124,130)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 11.3.3</span>

Did allowing a variable Higgs feature width significantly decrease the mass range?